# Print Runtime

In [1]:
#Sf1 = 6 million rows, sf10= 60 Millions rows
sf=10

In [2]:
from psutil import *
vCPU = str(cpu_count()) + " vCPU"
mem=round(virtual_memory().total/(1024 * 1024 * 1024),0)
runtime = 'Lineitem-SF'+str(sf)+' ,Google Colab, '+vCPU+' '+str(mem)+'GB'
runtime

'Lineitem-SF10 ,Google Colab, 16 vCPU 59.0GB'

# Install Packages

In [3]:
!pip install pyarrow --upgrade
!pip install duckdb --pre --upgrade
!pip install polars
!pip install datafusion==0.8
!curl https://clickhouse.com/ | sh
#could not install pyspark, something to do with java or some stuff in that nature

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 14.9/14.9 MB 90.3 MB/s eta 0:00:0000:0100:01
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 16.9/16.9 MB 81.4 MB/s eta 0:00:0000:0100:01
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 12.7/12.7 MB 71.3 MB/s eta 0:00:0000:010:01
  % Total    % Received % Xferd  Average Speed   Time    Time     Time  Current
                                 Dload  Upload   Total   Spent    Left  Speed
100  2739    0  2739    0     0  48052      0 --:--:-- --:--:-- --:--:-- 48052

Will download https://builds.clickhouse.com/master/amd64/clickhouse into clickhouse

  % Total    % Received % Xferd  Average Speed   Time    Time     Time  Current
                                 Dload  Upload   Total   Spent    Left  Speed
100  530M  100  530M    0     0  34.3M      0  0:00:15  0:00:15 --:--:-- 38.1M 0:00:06 37.9M

Successfully downloaded the ClickHouse binary, you can run it as:
    ./clickhouse

You can also install it:
    sudo ./clickhouse install


# Generate Parquet File

In [4]:
%%time
import duckdb
import pathlib


for x in range(0, sf) :
  con=duckdb.connect()
  con.sql('PRAGMA disable_progress_bar;SET preserve_insertion_order=false')
  con.sql(f"CALL dbgen(sf={sf} , children ={sf}, step = {x})") 
  for tbl in ['lineitem'] :
     pathlib.Path(f'./{tbl}').mkdir(parents=True, exist_ok=True) 
     con.sql(f"COPY (SELECT * FROM {tbl}) TO './{tbl}/{x}.parquet' ")
  con.close()

CPU times: user 5min 13s, sys: 20.2 s, total: 5min 33s
Wall time: 2min 42s


# DataFusion

In [5]:
%%time
from datafusion import SessionContext
ctx = SessionContext()
ctx.register_parquet('lineitem', './lineitem/*.parquet')
df = ctx.sql("select * from lineitem order by l_shipdate")
df.write_parquet("lineitem_Datafusion")

CPU times: user 2min 16s, sys: 20.9 s, total: 2min 36s
Wall time: 1min 6s


# DuckDB

In [6]:
%%time
import duckdb 
duckdb.sql(
'''
PRAGMA disable_progress_bar;
COPY  (select * from './lineitem/*.parquet' order by l_shipdate )
to 'lineitem_DuckDB' (FORMAT 'PARQUET', PER_THREAD_OUTPUT TRUE)
''')

CPU times: user 3min 56s, sys: 42.1 s, total: 4min 38s
Wall time: 29.1 s


# Clickhouse

In [7]:
%%time
#Query 1
!./clickhouse local --query=" \
    select * FROM file ('./lineitem/*.parquet',Parquet) \
    order by l_shipdate \
    INTO OUTFILE 'clickhouse.parquet' \
    FORMAT Parquet \
    "

CPU times: user 2.18 s, sys: 662 ms, total: 2.84 s
Wall time: 1min 38s


# Polars

In [8]:
%%time
import polars as pl
pl.scan_parquet("./lineitem/*.parquet").sort("l_shipdate").sink_parquet("lineitem_sort_polars.parquet")

STREAMING CHUNK SIZE: 3125 rows


CPU times: user 2min 48s, sys: 38.7 s, total: 3min 27s
Wall time: 1min 18s


# Pyarrow

In [9]:
%%time
import pyarrow.dataset as ds
lineitem = ds.dataset('./lineitem',format="parquet").sort_by('l_shipdate')
ds.write_dataset(lineitem,'./pyarrow', format="parquet")

CPU times: user 2min 52s, sys: 19.8 s, total: 3min 12s
Wall time: 2min 33s
